In [ ]:
import cv2 as cv
import numpy as np
from src.utils import *

In [ ]:
scale_imshow = lambda img: scaled_imshow(img,fx=0.4,fy=0.4)

In [ ]:
img = cv.imread('.\\data\\first_frame.png')
img = cv.resize(img,None,fx=0.4,fy=0.4)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

Detecting the dice tray. The dice tray is all black so a simple threshold should do the work

In [ ]:
#cv2.GaussianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),(7,7),0)
filtered = cv.bilateralFilter(gray, 9, 250, 250) 
#filtered = cv2.medianBlur(filtered,3)

scale_imshow(filtered)

#edges = cv2.Canny(filtered, 50, 60)
_,edges = cv.threshold(filtered,50,255,cv.THRESH_BINARY)
edges = 255-edges
#edges = cv2.morphologyEx(edges, cv2.MORPH_DILATE, np.ones((2,2)))

scale_imshow(edges)

contours,hierarchy = cv.findContours(edges, cv.RETR_TREE, 2)
i,largest_contour = max(enumerate(contours), key=lambda i_c:cv2.contourArea(i_c[1]))

# Draw the contours on the original image
img_contours = np.copy(img)
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)
cv2.drawContours(img_contours, [largest_contour], -1, (0, 0, 255), 2)

# Display the result
scale_imshow(img_contours)



Here we retrieve the dice from the contour hierarchy. The hierarchy used is *RETR_TREE*, which is described in [OpenCV docs](https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html). In short the hierarchy contains information about relations to other contours. It includes:
 - next contour (at the same height),
 - previous contour (at the same height),
 - child,
 - parent

Value of -1 indicates lack of another contour

In [ ]:
cropped = crop_contour(img,largest_contour)

imshow(cropped)

_,_,i_d1,_ = hierarchy[0,i]
i_d2,_,_,_ = hierarchy[0,i_d1]

dice1 = crop_contour(img,contours[i_d1])
dice2 = crop_contour(img,contours[i_d2])

imshow(dice1)
imshow(dice2)

The main board is not solid color, instead there is artwork drawn on it. In order to locate it descriptors might be used.

In [ ]:
board_ref = cv.imread('data\\board.jpg')
board_gray = cv.GaussianBlur(cv.cvtColor(board_ref, cv.COLOR_BGR2GRAY),(7,7),0)

scaled_imshow(board_ref)

In [ ]:
sift = cv.SIFT_create()
kp, desc = sift.detectAndCompute(gray, None)

kp2, desc2 = sift.detectAndCompute(board_gray, None)

In [ ]:
bf = cv.BFMatcher()
matches = bf.match(desc2, desc)

good_matches = sorted(matches, key=lambda x: x.distance)
_max =  max(matches, key=lambda x: x.distance).distance
good_matches = [m for m in matches if m.distance < 0.25 * _max]

src_pts = np.float32([kp2[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
M, mask = cv.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

warped_board = cv.warpPerspective(board_ref, M, (img.shape[1], img.shape[0]))
scale_imshow(warped_board)

result = cv.addWeighted(img, 1, warped_board, 0.5, 0)

scale_imshow(result)

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = mask.ravel().tolist(), # draw only inliers
                   flags = 2|4)
img3 = cv.drawMatches(board_ref,kp2,img,kp,good_matches,None,**draw_params)
scale_imshow(img3)

In [ ]:
warped_gray = cv.cvtColor(warped_board,cv.COLOR_BGR2GRAY)
_,edges2 = cv.threshold(warped_gray,50,255,cv.THRESH_BINARY)
edges2 = cv.morphologyEx(edges2,cv.MORPH_CLOSE,kernel=np.ones((7,7)))

scale_imshow(edges2)

contours,hierarchy = cv.findContours(edges2, cv.RETR_TREE, 2)
i,largest_contour2 = max(enumerate(contours), key=lambda i_c:cv2.contourArea(i_c[1]))

scale_imshow(crop_contour(img,largest_contour2))